# Assignment 3

In this assignment we are tasked with using SVD or ALS to create a recomender system.

Given the sparse nature of this matrix the `np.sparse.linalg.sparse()` function will not work.

We have a few options. most easily, we can just fill the sparse values with 0. This will introduce a lot of bias. Instead, it can be effective to replace the `na` values with the row/column means.

Finally, we can update the sparse matrix using a more sophsiticated algorithm. For example, we could use ALS to make a dense matrix and then do normal SVD to reduce the dimensions. However, we are still using imputed values to make it dense. A more interesting way of going about it is to use stochastic gradient decent to create two matrixies, P, Q that have dimensions of users/movies by `n_factors`. This is probably most elegant way to do it.

---

## Dataset

Here we are goign to use the `MovieLens` dataset 

In [1]:
import pandas as pd
import numpy as np
movies = pd.read_csv('~/DATA643/Project2/ml-latest-small/movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [2]:
ratings = pd.read_csv('~/DATA643/Project2/ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


We need to make this into a user/item dataframe first.

In [3]:
ratingsPiv = ratings.pivot(index='userId', columns='movieId', values='rating')

ratingsPiv.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


This is a very sparse matrix. We need to set the train/test split first before imputing values.

In [4]:
np.random.seed(101)
trainTestMask = np.random.choice([True, False], size=ratingsPiv.shape, p=[0.7, 0.3])
trainTestMask = pd.DataFrame(trainTestMask)
trainTestMask.index = ratingsPiv.index
trainTestMask.columns = ratingsPiv.columns
trainTestMask.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
userId,,,,,,,,,,,,,,,,,,,,,
1,True,True,True,True,True,False,True,False,False,True,...,True,True,False,True,True,True,False,True,True,True
2,False,True,True,True,False,False,True,True,False,True,...,True,True,False,True,False,True,True,True,True,True
3,False,True,True,True,False,True,True,True,False,True,...,True,True,True,True,False,True,False,True,False,False
4,True,False,False,True,True,False,False,True,True,False,...,False,True,True,True,False,True,False,True,True,True
5,False,True,True,False,True,False,True,True,True,True,...,False,True,True,True,True,True,True,True,True,True


In [5]:
train = pd.DataFrame(np.where(trainTestMask, ratingsPiv, np.NAN))
train.index = ratingsPiv.index
train.columns = ratingsPiv.columns

test = pd.DataFrame(np.where(np.invert(trainTestMask), ratingsPiv, np.NAN))
test.index = ratingsPiv.index
test.columns = ratingsPiv.columns

In [6]:
train.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


First Let's see the naieve RMSE:

In [7]:
testLong = test.copy()
testLong['userId'] = test.index
testLong = pd.melt(testLong, id_vars=['userId'])
testLong.head()

,userId,movieId,value
0,1,1,NaN
1,2,1,NaN
2,3,1,NaN
3,4,1,NaN
4,5,1,NaN


In [8]:
trainLong = train.copy()
trainLong['userId'] = train.index
trainLong = pd.melt(trainLong, id_vars=['userId'])
trainLong.head()

,userId,movieId,value
0,1,1,NaN
1,2,1,NaN
2,3,1,NaN
3,4,1,NaN
4,5,1,NaN


## Mean imputation

Here is implement an naieve algoritm that replaces na values with the mean of the dataset. 

In [9]:
colMean = trainLong.value.mean()
testLong['ratingMean'] = colMean

In [10]:
testLongMean = testLong[np.isfinite(testLong['value'])]
testLongMean.head()

,userId,movieId,value,ratingMean
6,7,1,3.0,3.542611
19,20,1,3.5,3.542611
43,44,1,4.0,3.542611
62,63,1,5.0,3.542611
68,69,1,5.0,3.542611


In [11]:
from sklearn.metrics import mean_squared_error
mean_squared_error(testLongMean.value, testLongMean.ratingMean)

1.1166519634936165

This uses the test set to validate the RMSE of just subing in the mean for all missing values. 

Over all it's not horrible.

This is the most baisic, implementation. It produces a reasonable RMSE.

To make this more complex we will use SVD on the matrix and set all `NA` values to the mean.

This matrix has zeros for `na` values and will produce a biaed result.

In [12]:
import scipy
from scipy.sparse.linalg import svds
svdMean = train.fillna(colMean)
svdMean = svdMean.as_matrix()

In [13]:
u,s,v = svds(svdMean, k=100)

In [14]:
print(u.shape, s.shape, v.shape)

(671, 100) (100,) (100, 9066)


In [15]:
S = np.diag(s) # To make it a diagonal matrix

In [16]:
matMean = u.dot(S).dot(v)
#pd.DataFrame(matZeros).head()
matMean.shape

(671, 9066)

In [17]:
dfMean = pd.DataFrame(matMean)
dfMean.index = ratingsPiv.index
dfMean.columns = ratingsPiv.columns
dfMean['userId'] = dfMean.index
dfMean.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,161155,161594,161830,161918,161944,162376,162542,162672,163949,userId
userId,,,,,,,,,,,,,,,,,,,,,
1,3.719375,3.521190,3.558313,3.534645,3.545526,3.539794,3.526117,3.529424,3.541760,3.572198,...,3.544811,3.546439,3.548524,3.547312,3.543764,3.535162,3.542359,3.540578,3.540815,1
2,3.582762,3.528446,3.585822,3.543436,3.593353,3.626788,3.438101,3.548813,3.528265,3.575843,...,3.548080,3.545952,3.550197,3.548830,3.525873,3.538473,3.543247,3.542746,3.548786,2
3,3.531427,3.507829,3.512722,3.512156,3.542782,3.605956,3.570752,3.551656,3.535466,3.528208,...,3.531653,3.543158,3.549639,3.548190,3.526202,3.540696,3.542268,3.539899,3.538938,3
4,3.738463,3.369270,3.606657,3.561323,3.399356,3.290927,3.596625,3.498572,3.477923,3.738675,...,3.536440,3.540866,3.546232,3.545798,3.568692,3.549601,3.544026,3.545135,3.540273,4
5,3.601069,3.695016,3.659219,3.447812,3.548668,3.488521,3.600830,3.584020,3.526897,3.662955,...,3.538419,3.541772,3.536980,3.537978,3.540289,3.542556,3.542056,3.539241,3.544245,5


In [18]:

meanSeries = pd.melt(dfMean, id_vars=['userId'])
meanSeries.head()

,userId,movieId,value
0,1,1,3.719375
1,2,1,3.582762
2,3,1,3.531427
3,4,1,3.738463
4,5,1,3.601069


In [19]:
meanSvdEval = pd.merge(testLong, meanSeries,  
                  how='left', 
                  left_on=['userId','movieId'], 
                  right_on = ['userId','movieId'])
meanSvdEval = meanSvdEval[np.isfinite(meanSvdEval['value_x'])]

meanSvdEval.head()

,userId,movieId,value_x,ratingMean,value_y
6,7,1,3.0,3.542611,3.771932
19,20,1,3.5,3.542611,3.455594
43,44,1,4.0,3.542611,3.477622
62,63,1,5.0,3.542611,4.041213
68,69,1,5.0,3.542611,3.941167


In [20]:
mean_squared_error(meanSvdEval['value_x'], meanSvdEval['value_y'])

1.0729859682025999

This produces a better RMSE on the test set. Infact, it's down right impressive. This is actually a bit alarming since it performs better than the cross validated surprise package below. 

### Suprpise Package
Using the surprise package we can do SVD with a sparse matrix:

In [21]:
import surprise
from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise import SVD
from surprise.model_selection import train_test_split
from surprise import accuracy
from surprise.model_selection import cross_validate

df = pd.read_csv('/Users/kailukowiak/DATA643/Project2/ml-latest-small/ratings.csv')
reader = Reader(rating_scale=(1, 5))


data = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)
#algo = SVD()
mod = cross_validate(algo, data, measures=['RMSE'], cv=5, verbose=True)


trainset, testset = train_test_split(data, test_size=.3)

# We'll use the famous SVD algorithm.
#algo = SVD()
algo = surprise.prediction_algorithms.matrix_factorization.SVD()
mod = cross_validate(algo, data, measures=['RMSE'], cv=5, verbose=True)
# Train the algorithm on the trainset, and predict ratings for the testset
#algo.fit(trainset)
#predictions = algo.test(testset)

# Then compute RMSE
#accuracy.rmse(predictions)

NameError: name 'algo' is not defined

This uses stochastic gradient decent to figure out the ratings and then computes SVD on that. The mean RMSE is 0.893 which is worrying because I would have presumed that this package would perform better than my mean imputed SVD algorithm.

Let's do a quick sanity check to see what we are recomending. UserID of 6 is the first person in our test set.

In [ ]:
user6 = ratings[ratings['userId']==6][['userId', 'movieId', 'rating']]
user6test = pd.merge(user6, movies, how='left',
                    left_on='movieId',
                    right_on='movieId')

user6test.sort_values(by=['rating'], ascending=False).iloc[0:10,:]

In [ ]:
predFavMovies6 = meanSeries[meanSeries['userId']==6].sort_values(by='value', ascending=False)
user6meanSVD = pd.merge(predFavMovies6, movies, on='movieId')
user6meanSVD.head(10)

This is a pretty valid prediction for user 6.

All told, the movies don't seem out of line with what the user rated on their own.

using surprise:

In [ ]:
PU = algo.pu
QI = algo.qi
surprisePreds = PU.dot(QI.T)

In [ ]:
QI.shape

In [ ]:
ratings.movieId.unique().size

This is a smaller dataset which warents invistigation, unfortunetly, I am running out of time, so I'm just going to show the predictions.

I believe the problem stems from the training/test set only having a subset of the movies, thus making a smaller matrix. This is a bug in an otherwise great library. If I had more time, I'd investigate further.

In [ ]:
8401/9066

In [ ]:
surpriseDF = pd.DataFrame(surprisePreds)
surpriseDF.index = ratingsPiv.index


In [ ]:
algo.predict(iid='2', uid='1')

In [ ]:
scipy.sparse.coo_matrix(ratingsMatZero)


In [ ]:
import implicit
# initialize a model
model = implicit.als.AlternatingLeastSquares(factors=50)
#model.fit(scipy.sparse.coo_matrix(ratingsMat))

item_user_data = scipy.sparse.coo_matrix((ratings['rating'].values, (ratings['movieId'].values, ratings['userId'].values)))
# train the model on a sparse matrix of item/user/confidence weights
model.fit(item_user_data)

In [ ]:
print(UF.shape, IF.shape)

In [ ]:
scipy.sparse.coo_matrix(ratingsMatZero).toarray().shape

In [ ]:
model = implicit.als.AlternatingLeastSquares(factors=100)
#model.fit(scipy.sparse.coo_matrix(ratingsMat))

item_user_data = scipy.sparse.coo_matrix(ratingsMatZero.T)
model.fit(item_user_data)

In [ ]:
UF = model.user_factors
IF = model.item_factors


In [ ]:
IF.shape

In [ ]:
implicitPreds = UF.dot(IF.T)

In [ ]:
implicitPreds.shape

This is correct, but there are some very weird answers including negatives and numbers well above 5. I'll fix these and then evaluate.

In [ ]:
implicitDF = pd.DataFrame(implicitPreds)
implicitDF.index = ratingsPiv.index
implicitDF.columns = ratingsPiv.columns
implicitDF.head()

In [ ]:
implicitDF[implicitDF < 0] = 0
implicitDF[implicitDF > 5] = 5

In [ ]:
implicitDF.head()

In [ ]:

implicitDF['userId'] = implicitDF.index
implicitSeries = pd.melt(implicitDF, id_vars=['userId'])
implicitSeries.head()


In [ ]:
implicitEval = pd.merge(testLong, implicitSeries,  
                  how='left', 
                  left_on=['userId','movieId'], 
                  right_on = ['userId','movieId'])
implicitEval = meanSvdEval[np.isfinite(meanSvdEval['value_x'])]

implicitEval.head()

In [ ]:
mean_squared_error(implicitEval['value_x'], implicitEval['value_y'])

This is actually a pretty good algorithm too. However, I ran out of time, so I didn't have the chance to use the train/test split. 

In [ ]:
predFavMovies6 = implicitSeries[implicitSeries['userId']==6].sort_values(by='value', ascending=False)
user6meanSVD = pd.merge(predFavMovies6, movies, on='movieId')
user6meanSVD.head(10)

These predictions are different, but I wouldn't say worse.

## Further Notes

I was time constrainted on this assignmet. Idealy one should set the SVD to only lose 80-90% of the energy of the original. I'm pretty confident that I achieved this anyways, given the larege number or movies, but if I did lose more, it would be preferable to reduce the number of factors.